In [23]:
import pandas as pd
import numpy as np
import nltk
import re
import textblob 
import enchant
import string
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from spellchecker import SpellChecker
from nltk.corpus import stopwords
from textblob import TextBlob
from googletrans import Translator
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import time

In [24]:
np.random.seed(500)

In [25]:
dictionary=enchant.Dict("en_US")
translator=Translator()
lemmatizer=WordNetLemmatizer()
spell=SpellChecker()

In [26]:
import sys

In [27]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [28]:
Corpus = pd.read_csv(r"C:\Users\Dell\Desktop\Dataset.csv",encoding='latin-1')

In [29]:
dataset=Corpus.copy()

In [30]:
dataset.head()

,Reviews,Label
0,build quality towards their performance runnin...,-1
1,I wouldn't recommend it. the speaker performa...,0
2,takes good performance images. I am very happy,1
3,it can take high performance pictures,1
4,it has to do with the high performance,1


In [31]:
printable = set(string.printable)
dataset['cleantext'] = Corpus['Reviews'].apply(lambda row: ''.join(filter(lambda x:x in printable,row)))

In [32]:
dataset['cleantext']= [word_tokenize(entry) for entry in dataset['cleantext']]

In [33]:
dataset['cleantext']= [pos_tag(entry) for entry in dataset['cleantext']]

In [34]:
REPLACE = re.compile('[.;:!\'$?,\"()\[\]]')

In [35]:
dataset.head()

,Reviews,Label,cleantext
0,build quality towards their performance runnin...,-1,"[(build, JJ), (quality, NN), (towards, NNS), (..."
1,I wouldn't recommend it. the speaker performa...,0,"[(I, PRP), (would, MD), (n't, RB), (recommend,..."
2,takes good performance images. I am very happy,1,"[(takes, VBZ), (good, JJ), (performance, NN), ..."
3,it can take high performance pictures,1,"[(it, PRP), (can, MD), (take, VB), (high, JJ),..."
4,it has to do with the high performance,1,"[(it, PRP), (has, VBZ), (to, TO), (do, VB), (w..."


In [36]:
def pre_process(Reviews):
    # lowercase
    Reviews = Reviews.lower()
    # tags
    Reviews = re.sub('&lt;/?.*?&gt;',' &lt;&gt; ',Reviews)
    # special characters and digits
    Reviews=re.sub('(\\d|\\W)+',' ',Reviews)
    # remove punctuation
    Reviews = re.sub('[.;:!\'?,\"()\[\]]', '', Reviews)
    #Reviews = [REPLACE.sub('', line) for line in Reviews]
    
    return Reviews

dataset['cleantext'] = Corpus['Reviews'].apply(lambda x:pre_process(x))

In [37]:
  dataset.head()

,Reviews,Label,cleantext
0,build quality towards their performance runnin...,-1,build quality towards their performance runnin...
1,I wouldn't recommend it. the speaker performa...,0,i wouldn t recommend it the speaker performan...
2,takes good performance images. I am very happy,1,takes good performance images i am very happy
3,it can take high performance pictures,1,it can take high performance pictures
4,it has to do with the high performance,1,it has to do with the high performance


In [38]:
dataset.head()

,Reviews,Label,cleantext
0,build quality towards their performance runnin...,-1,build quality towards their performance runnin...
1,I wouldn't recommend it. the speaker performa...,0,i wouldn t recommend it the speaker performan...
2,takes good performance images. I am very happy,1,takes good performance images i am very happy
3,it can take high performance pictures,1,it can take high performance pictures
4,it has to do with the high performance,1,it has to do with the high performance


In [39]:
from nltk.corpus import stopwords


In [40]:
english_stop_words = stopwords.words('english')
#english_stop_words = ENGLISH_STOP_WORDS
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words]) 
        )
    return removed_stop_words

dataset['cleantext'] = remove_stop_words(dataset['cleantext'])

In [41]:
print(dataset['Reviews'][10])
print(dataset['cleantext'][10])

 Excellent display performance. It runs fast.
excellent display performance runs fast


In [42]:
from nltk.stem.porter import PorterStemmer

def get_stemmed_text(Corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in Reviews.split()]) for Reviews in Corpus]

dataset['text'] = get_stemmed_text(dataset['cleantext'])

In [43]:
dataset.head()

,Reviews,Label,cleantext,text
0,build quality towards their performance runnin...,-1,build quality towards performance running slow,build qualiti toward perform run slow
1,I wouldn't recommend it. the speaker performa...,0,recommend speaker performance normal,recommend speaker perform normal
2,takes good performance images. I am very happy,1,takes good performance images happy,take good perform imag happi
3,it can take high performance pictures,1,take high performance pictures,take high perform pictur
4,it has to do with the high performance,1,high performance,high perform


In [44]:
for name, df in dataset.items():
    # Saving to file
    df.to_csv('Preprocessed data.csv', index=False)